## Import thư viện

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow_addons as tfa
from kaggle_datasets import KaggleDatasets

## Load Data

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
EPOCHS = 10
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
WIDTH = 768
HEIGHT = 768
CHANNELS = 3
LEARNING_RATE = 0.001
CLASSES = 6
SEED = 32
top_dropout_rate = 0.2

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('fgvc8aug')
TRAIN_PATH = GCS_DS_PATH + "/data_full_augmentation_images/data_full_augmentation/images/"
print(GCS_DS_PATH)

In [ ]:
final_model = 'FGVC8-VGG19.h5'
hist_path = 'FGVC8-VGG19.log'
train_image = '../input/fgvc8aug/data_full_augmentation_images/data_full_augmentation/images'
train_df = pd.read_csv('../input/fgvc8aug/data.csv', )

In [ ]:
train_df = train_df[["image", "labels"]]
mlb = MultiLabelBinarizer().fit(train_df.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mlb.transform(train_df.labels.apply(lambda x : x.split())), columns = mlb.classes_)

labels = pd.concat([train_df['image'], labels], axis=1)
labels.head()

In [ ]:
def format_path(st):
    return TRAIN_PATH + st

train_paths = labels.image.apply(format_path).values

train_labels = np.float32(labels.loc[:, 'complex':'scab'].values)
train_paths, valid_paths, train_labels, valid_labels =\
train_test_split(train_paths, train_labels, test_size=0.15, random_state=2020)

In [ ]:
def process_img(filepath,label):
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=CHANNELS)
    image = tf.image.convert_image_dtype(image, tf.float32) 
    image = tf.image.resize(image, [HEIGHT,WIDTH])
    return image,label

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(process_img, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(process_img, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

## Get model

In [ ]:
def get_model():
    VGG19_MODEL = tf.keras.applications.VGG19(weights='imagenet' ,include_top=False, input_shape=(HEIGHT, WIDTH, 3))
    
    x=VGG19_MODEL.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(256,activation='relu')(x)
    x=Dropout(0.2)(x)
    x=Dense(128,activation='relu')(x)
    prediction=Dense(6,activation='sigmoid')(x)

    model=Model(inputs=VGG19_MODEL.input, outputs=prediction)
    
    return model

In [ ]:
with strategy.scope():
    model = get_model()

    f1_score = tfa.metrics.F1Score(num_classes=6, threshold=0.4, average='micro')
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.0005) , loss='binary_crossentropy', metrics=[f1_score, 'accuracy'])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, dpi=60)

## Trainning

In [ ]:
checkpoint = ModelCheckpoint(
    final_model,
    monitor = 'val_accuracy',
    mode = 'max',
    save_best_only = True,
    save_weights_only= False ,
    perior = 1,
    verbose = 1
)

early_stopping = EarlyStopping(
    monitor = 'val_accuracy',
    mode = 'auto',
    min_delta = 0.0001,
    patience = 3,
    baseline = None,
    restore_best_weights = True,
    verbose = 1
)
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

In [ ]:
lrfn = build_lrfn()
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

In [ ]:
params = model.fit(
    train_dataset, 
    validation_data = valid_dataset, 
    epochs = EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks = [lr_schedule, early_stopping, checkpoint, CSVLogger(hist_path)]
)